# Save NIDM-Results packs to NiMARE dataset

TODO:
- Create second dataset with local maxima.

In [1]:
import json
from glob import glob
import nibabel as nib
import pandas as pd
import numpy as np
from scipy import ndimage
from os.path import basename, join, isfile

In [2]:
def _get_val(row, input_arr):
    i, j, k = row
    return input_arr[i, j, k]


def _local_max(data, min_distance):
    """Find all local maxima of the array, separated by at least min_distance.
    Adapted from https://stackoverflow.com/a/22631583/2589328
    Parameters
    ----------
    data : array_like
        3D array of with masked values for cluster.
    min_distance : :obj:`int`
        Minimum distance between local maxima in ``data``, in terms of
        voxels (not mm).
    Returns
    -------
    ijk : :obj:`numpy.ndarray`
        (n_foci, 3) array of local maxima indices for cluster.
    vals : :obj:`numpy.ndarray`
        (n_foci,) array of values from data at ijk.
    """
    data_max = ndimage.filters.maximum_filter(data, min_distance)
    maxima = (data == data_max)
    data_min = ndimage.filters.minimum_filter(data, min_distance)
    diff = ((data_max - data_min) > 0)
    maxima[diff == 0] = 0

    labeled, num_objects = ndimage.label(maxima)
    ijk = np.array(ndimage.center_of_mass(data, labeled, range(1, num_objects+1)))
    ijk = np.round(ijk).astype(int)

    vals = np.apply_along_axis(arr=ijk, axis=1, func1d=_get_val, input_arr=data)
    order = (-vals).argsort()
    vals = vals[order]
    ijk = ijk[order, :]

    # Reduce list of subpeaks based on distance
    keep_idx = np.ones(ijk.shape[0]).astype(bool)
    for i in range(ijk.shape[0]):
        for j in range(i+1, ijk.shape[0]):
            if keep_idx[i] == 1:
                dist = np.linalg.norm(ijk[i, :] - ijk[j, :])
                keep_idx[j] = dist > min_distance
    ijk = ijk[keep_idx, :]
    vals = vals[keep_idx]
    return ijk, vals

In [3]:
f1 = '/Users/tsalo/Documents/tsalo/NiMARE/nimare/tests/data/nidm_pain_dset.json'
f2 = '/Users/tsalo/Documents/tsalo/NiMARE/nimare/tests/data/nidm_pain_dset_with_subpeaks.json'

In [4]:
ddict = {}
folders = sorted(glob('/Users/tsalo/Downloads/nidm-pain-results/pain_*.nidm'))
for folder in folders:
    name = basename(folder)
    ddict[name] = {}
    ddict[name]['contrasts'] = {}
    ddict[name]['contrasts']['1'] = {}
    ddict[name]['contrasts']['1']['coords'] = {}
    ddict[name]['contrasts']['1']['coords']['space'] = 'MNI'
    ddict[name]['contrasts']['1']['images'] = {}
    ddict[name]['contrasts']['1']['images']['space'] = 'MNI_2mm'
    # con file
    files = glob(join(folder, 'Contrast*.nii.gz'))
    files = [f for f in files if 'StandardError' not in basename(f)]
    if files:
        f = sorted(files)[0]
    else:
        f = None
    ddict[name]['contrasts']['1']['images']['con'] = f
    # se file
    files = glob(join(folder, 'ContrastStandardError*.nii.gz'))
    if files:
        f = sorted(files)[0]
    else:
        f = None
    ddict[name]['contrasts']['1']['images']['se'] = f
    # z file
    files = glob(join(folder, 'ZStatistic*.nii.gz'))
    if files:
        f = sorted(files)[0]
    else:
        f = None
    ddict[name]['contrasts']['1']['images']['z'] = f
    # t file
    # z file
    files = glob(join(folder, 'TStatistic*.nii.gz'))
    if files:
        f = sorted(files)[0]
    else:
        f = None
    ddict[name]['contrasts']['1']['images']['t'] = f
    # sample size
    f = join(folder, 'DesignMatrix.csv')
    if isfile(f):
        df = pd.read_csv(f, header=None)
        n = [df.shape[0]]
    else:
        n = None
    ddict[name]['contrasts']['1']['sample_sizes'] = n
    # foci
    files = glob(join(folder, 'ExcursionSet*.nii.gz'))
    f = sorted(files)[0]
    img = nib.load(f)
    data = np.nan_to_num(img.get_data())
    # positive clusters
    binarized = np.copy(data)
    binarized[binarized>0] = 1
    binarized[binarized<0] = 0
    binarized = binarized.astype(int)
    labeled = ndimage.measurements.label(binarized, np.ones((3, 3, 3)))[0]
    clust_ids = sorted(list(np.unique(labeled)[1:]))
    ijk = np.hstack([np.where(data * (labeled == c) == np.max(data * (labeled == c))) for c in clust_ids])
    ijk = ijk.T
    xyz = nib.affines.apply_affine(img.affine, ijk)
    ddict[name]['contrasts']['1']['coords']['x'] = list(xyz[:, 0])
    ddict[name]['contrasts']['1']['coords']['y'] = list(xyz[:, 1])
    ddict[name]['contrasts']['1']['coords']['z'] = list(xyz[:, 2])

with open(f1, 'w') as fo:
    json.dump(ddict, fo, sort_keys=True, indent=4)

In [5]:
ddict = {}
folders = sorted(glob('/Users/tsalo/Downloads/nidm-pain-results/pain_*.nidm'))
for folder in folders:
    name = basename(folder)
    ddict[name] = {}
    ddict[name]['contrasts'] = {}
    ddict[name]['contrasts']['1'] = {}
    ddict[name]['contrasts']['1']['coords'] = {}
    ddict[name]['contrasts']['1']['coords']['space'] = 'MNI'
    ddict[name]['contrasts']['1']['images'] = {}
    ddict[name]['contrasts']['1']['images']['space'] = 'MNI_2mm'
    # con file
    files = glob(join(folder, 'Contrast*.nii.gz'))
    files = [f for f in files if 'StandardError' not in basename(f)]
    if files:
        f = sorted(files)[0]
    else:
        f = None
    ddict[name]['contrasts']['1']['images']['con'] = f
    # se file
    files = glob(join(folder, 'ContrastStandardError*.nii.gz'))
    if files:
        f = sorted(files)[0]
    else:
        f = None
    ddict[name]['contrasts']['1']['images']['se'] = f
    # z file
    files = glob(join(folder, 'ZStatistic*.nii.gz'))
    if files:
        f = sorted(files)[0]
    else:
        f = None
    ddict[name]['contrasts']['1']['images']['z'] = f
    # t file
    # z file
    files = glob(join(folder, 'TStatistic*.nii.gz'))
    if files:
        f = sorted(files)[0]
    else:
        f = None
    ddict[name]['contrasts']['1']['images']['t'] = f
    # sample size
    f = join(folder, 'DesignMatrix.csv')
    if isfile(f):
        df = pd.read_csv(f, header=None)
        n = [df.shape[0]]
    else:
        n = None
    ddict[name]['contrasts']['1']['sample_sizes'] = n
    # foci
    files = glob(join(folder, 'ExcursionSet*.nii.gz'))
    f = sorted(files)[0]
    img = nib.load(f)
    data = np.nan_to_num(img.get_data())
    # positive clusters
    binarized = np.copy(data)
    binarized[binarized>0] = 1
    binarized[binarized<0] = 0
    binarized = binarized.astype(int)
    labeled = ndimage.measurements.label(binarized, np.ones((3, 3, 3)))[0]
    clust_ids = sorted(list(np.unique(labeled)[1:]))
    ijk = []
    for c in clust_ids:
        masked_data = data * (labeled == c)
        voxel_size = np.prod(img.header.get_zooms())
        min_dist = int(np.round(8. / np.power(voxel_size, 1./3.)))
        subpeak_ijk, subpeak_vals = _local_max(masked_data, min_distance=min_dist)
        n_subpeaks = np.min((len(subpeak_vals), 4))
        subpeak_ijk = subpeak_ijk[:n_subpeaks, :]
        ijk.append(subpeak_ijk)
    ijk = np.vstack(ijk)
    xyz = nib.affines.apply_affine(img.affine, ijk)
    ddict[name]['contrasts']['1']['coords']['x'] = list(xyz[:, 0])
    ddict[name]['contrasts']['1']['coords']['y'] = list(xyz[:, 1])
    ddict[name]['contrasts']['1']['coords']['z'] = list(xyz[:, 2])

with open(f2, 'w') as fo:
    json.dump(ddict, fo, sort_keys=True, indent=4)